In [1]:
import pandas as pd
import geopandas as gp 
import json
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gp
sns.set(rc={'figure.figsize':(12,8)})
%matplotlib inline
plt.style.use('default')
%matplotlib inline
from tqdm import tqdm
import glob
import numpy as np
from scipy.stats import pearsonr
from scipy.stats import linregress

In [2]:
files = glob.glob('../BL_Work/openPrescribe/savings/*.h5')
files.sort()
print files

['../BL_Work/openPrescribe/savings/201008.h5', '../BL_Work/openPrescribe/savings/201009.h5', '../BL_Work/openPrescribe/savings/201010.h5', '../BL_Work/openPrescribe/savings/201011.h5', '../BL_Work/openPrescribe/savings/201012.h5', '../BL_Work/openPrescribe/savings/201101.h5', '../BL_Work/openPrescribe/savings/201102.h5', '../BL_Work/openPrescribe/savings/201103.h5', '../BL_Work/openPrescribe/savings/201104.h5', '../BL_Work/openPrescribe/savings/201105.h5', '../BL_Work/openPrescribe/savings/201106.h5', '../BL_Work/openPrescribe/savings/201107.h5', '../BL_Work/openPrescribe/savings/201108.h5', '../BL_Work/openPrescribe/savings/201109.h5', '../BL_Work/openPrescribe/savings/201110.h5', '../BL_Work/openPrescribe/savings/201111.h5', '../BL_Work/openPrescribe/savings/201112.h5', '../BL_Work/openPrescribe/savings/201201.h5', '../BL_Work/openPrescribe/savings/201202.h5', '../BL_Work/openPrescribe/savings/201203.h5', '../BL_Work/openPrescribe/savings/201204.h5', '../BL_Work/openPrescribe/savings

In [3]:
# % ls ../BL_Work/openPrescribe/savings/

In [4]:
files[20]

'../BL_Work/openPrescribe/savings/201204.h5'

In [5]:
pdp = pd.read_hdf('../BL_Work/openPrescribe/savings/201008.h5',compression='gzip')
pdp.dropna()
for month in [ '201009' , '201010']:
    file_path  = '../BL_Work/openPrescribe/savings/' + month + '.h5'
#     pdp2 = pd.read_csv(file_path,compression='gzip')
    pdp2 = pd.read_hdf(file_path,compression='gzip')
    pdp2.dropna()
    pdp = pdp.append(pdp2)

In [6]:
len(pdp)

25553376

In [7]:
pdp.head()

Unnamed: 0    0    1       2                3  \
0 8327         9404  Q30  5D7  A86011  0101010C0AAAAAA   
  35930       38180  Q30  5D7  A86040  0101010C0BBAAAA   
  60856       65924  Q30  5D8  A87016  0101010C0AAAAAA   
  64926       69994  Q30  5D8  A87020  0101010C0AAAAAA   
  69494       74562  Q30  5D8  A87029  0101010C0BBAAAA   

                                                4    5     6     7      8  \
0 8327   Alum Hydrox_Cap 475mg                     1.0  5.81  5.38  186.0   
  35930  Alu-Cap_Cap 475mg                         2.0  7.50  6.97  240.0   
  60856  Alum Hydrox_Cap 475mg                     1.0  5.25  4.86  168.0   
  64926  Alum Hydrox_Cap 475mg                     1.0  3.75  3.48  120.0   
  69494  Alu-Cap_Cap 475mg                         1.0  3.75  3.48  120.0   

         ...                   14     15         16    17   18    19  20  \
0 8327   ...  Aluminium Hydroxide  475.0  0101010C0   9.3  1.0   9.3  AA   
  35930  ...  Aluminium Hydroxide  475.0  0101010C0  12.0  1.0  12.0  BB   
  60856  ...  Aluminium Hydroxide  475.0  0101010C0   8.4  1.0   8.4  AA   
  64926  ...  Aluminium Hydroxide  475.0  0101010C0   6.0  1.0   6.0  AA   
  69494  ...  Aluminium Hydroxide  475.0  0101010C0   6.0  1.0   6.0  BB   

           21      22     23  
0 8327   0.03  0.0305  0.093  
  35930  0.03  0.0305  0.120  
  60856  0.03  0.0305  0.084  
  64926  0.03  0.0305  0.060  
  69494  0.03  0.0305  0.060  

[5 rows x 25 columns]

In [8]:
# fentanyl = pdp[pdp['16'].str.contains('0309010C0')]
# # fentanyl[fentanyl['20'] != 'AA']
# fentanyl

In [9]:
%store -r opiod_BNF
# %store -r antiDep_BNF

In [10]:
# opiod_BNF = antiDep_BNF
# opiod_BNF

In [11]:
LSOA_dist = json.load(open('scratchpad/GP_LSOA_PATIENTSDIST.json','rb'))

In [12]:
opiod_BNF.head()

,Unnamed: 0,BNF_code,NAME,UNII_drugbank
88,88,0104020D0,Codeine Phosphate Compound Mixtures ...,UX6OWY2V7J
90,90,0104020L0,Loperamide Hydrochloride ...,6X9OC3H4II
91,91,0104020N0,Opium & Morphine ...,76I7G6D29C
92,92,0104020P0,Loperamide Hydrochloride & Simeticone ...,6X9OC3H4II
154,154,0106060A0,Methylnaltrexone Bromide ...,0RK7M7IABE


In [13]:
ward_pop = pd.read_csv('scratchpad/ward_pop.csv')

/home/sagar/anaconda3/envs/env_py2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (23,24,25,26,27,28,29,30,31,32,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
GP_META = pd.read_csv('../BL_Work/epraccur.csv',header=None)
GP_META.rename(columns={0:'BP_code',
                        1:'Name',
                        2:'Grouping',
                        3:'National_geo',
                        4:'Addr1',
                        5:'Addr2',
                        6:'Addr3',
                        7: 'Addr4',
                        8:'Addr5',
                        9:'Postcode',
                        10:'Open',
                        11:'Closed',
                        12:'Status',
                        13:'Org type code',
                        14:'Commissioner',
                        15:'Join provider',
                        16:'Left provider',
                        17:'Contact',
                        18:'Null1',
                        19:'Null2',
                        20:'Null3',
                        21:'Amended',
                        22:'Null4',
                        23:'Provider',
                        24:'Null5',
                        25:'Setting',
                        26:'Null6'}, 
                 inplace=True)

In [15]:
Open_filtered_GPs = {}
for index,row in GP_META.iterrows():
    if row ['Setting'] == 4 and row['Status'] =='A':
        Open_filtered_GPs[row['BP_code']] = row['Postcode'].strip()

In [16]:
population = {}
for index, row in ward_pop.iterrows():
    population[row['Ward Code 1']] = float(row['All Ages'].replace(',',''))

In [17]:
population_borough = {}
for name,group in ward_pop.groupby('Local Authority'):
    population_borough[name] = 0.0
    for index , row in group.iterrows():
        population_borough[name] += float(row['All Ages'].replace(',',''))

In [18]:
pdp.head()

Unnamed: 0    0    1       2                3  \
0 8327         9404  Q30  5D7  A86011  0101010C0AAAAAA   
  35930       38180  Q30  5D7  A86040  0101010C0BBAAAA   
  60856       65924  Q30  5D8  A87016  0101010C0AAAAAA   
  64926       69994  Q30  5D8  A87020  0101010C0AAAAAA   
  69494       74562  Q30  5D8  A87029  0101010C0BBAAAA   

                                                4    5     6     7      8  \
0 8327   Alum Hydrox_Cap 475mg                     1.0  5.81  5.38  186.0   
  35930  Alu-Cap_Cap 475mg                         2.0  7.50  6.97  240.0   
  60856  Alum Hydrox_Cap 475mg                     1.0  5.25  4.86  168.0   
  64926  Alum Hydrox_Cap 475mg                     1.0  3.75  3.48  120.0   
  69494  Alu-Cap_Cap 475mg                         1.0  3.75  3.48  120.0   

         ...                   14     15         16    17   18    19  20  \
0 8327   ...  Aluminium Hydroxide  475.0  0101010C0   9.3  1.0   9.3  AA   
  35930  ...  Aluminium Hydroxide  475.0  0101010C0  12.0  1.0  12.0  BB   
  60856  ...  Aluminium Hydroxide  475.0  0101010C0   8.4  1.0   8.4  AA   
  64926  ...  Aluminium Hydroxide  475.0  0101010C0   6.0  1.0   6.0  AA   
  69494  ...  Aluminium Hydroxide  475.0  0101010C0   6.0  1.0   6.0  BB   

           21      22     23  
0 8327   0.03  0.0305  0.093  
  35930  0.03  0.0305  0.120  
  60856  0.03  0.0305  0.084  
  64926  0.03  0.0305  0.060  
  69494  0.03  0.0305  0.060  

[5 rows x 25 columns]

In [19]:
# rantadine = pdp.loc[pdp['16'] == '0103010T0']

In [20]:
# rantadine[rantadine['15']==150]

In [21]:
def func_ome(df,drugBNF,ome_map):
    df['presc_ome'] = df['8'] *df['15']*ome_map[drugBNF]
    return df

def calculateOME(pdp,ome_map):
    pdp['presc_ome'] = 0.0
    return pdp.groupby('3',as_index=False).apply(lambda df: func_ome(df , df.name, ome_map ))

%store -r ome_map

opioids = pdp[pdp['3'].isin(ome_map.keys())]
opioids = calculateOME(opioids , ome_map) 

/10TBdrive/sagarj/anaconda3/envs/env_py2/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [22]:
ome_map

{'040702020AAAAAA': 2.0,
 '040702020AAABAB': 2.0,
 '040702020BBAAAA': 2.0,
 '040702020BBABAB': 2.0,
 '040702040AAAAAA': 0.1,
 '040702040AAABAB': 0.1,
 '040702040AAACAC': 0.1,
 '040702040AAADAD': 0.1,
 '040702040AAAEAE': 0.1,
 '040702040AAAFAF': 0.1,
 '040702040AAAGAG': 0.1,
 '040702040AAAHAH': 0.1,
 '040702040AAAIAI': 0.1,
 '040702040AAAJAJ': 0.1,
 '040702040AAAMAM': 0.1,
 '040702040AAANAN': 0.1,
 '040702040AAAPAP': 0.1,
 '040702040AAATAT': 0.1,
 '040702040AAAUAU': 0.1,
 '040702040AAAVAV': 0.1,
 '040702040AAAWAW': 0.1,
 '040702040AAAXAX': 0.1,
 '040702040AAAYAY': 0.1,
 '040702040AAAZAZ': 0.1,
 '040702040AABABA': 0.1,
 '040702040AABBBB': 0.1,
 '040702040AABCBC': 0.1,
 '040702040AABDBD': 0.1,
 '040702040BBAAAA': 0.1,
 '040702040BBABAB': 0.1,
 '040702040BBACAC': 0.1,
 '040702040BBADAD': 0.1,
 '040702040BBAEAE': 0.1,
 '040702040BBAFAF': 0.1,
 '040702040BBAGAD': 0.1,
 '040702040BBAHAE': 0.1,
 '040702040BBAIAM': 0.1,
 '040702040BBAJAN': 0.1,
 '040702040BBAKAY': 0.1,
 '040702040BCAAAA': 0.1,


In [23]:
len(opioids)

660568

In [24]:
# opioids = pdp.loc[pdp['16'].isin(opiod_BNF['BNF_code'])]

In [25]:
non_generic_opioid = opioids.loc[opioids['20']!='AA']

In [26]:
sum(non_generic_opioid['23'])

2899575.387807917

In [27]:
sum(opioids['7'])

54657651.43999427

In [28]:
len(non_generic_opioid), len(opioids)

(211824, 660568)

In [29]:
GP_postcodes_long = {}
for name , group in non_generic_opioid.groupby('2'):
    postcode = list(set(group['10']))
    GP_postcodes_long[name] = str(postcode[0]).strip()

In [30]:
ward_df = pd.read_csv('../BL_Work/pcd11_par11_wd11_lad11_ew_lu.csv')

# ward_df.head()

ward_postcode_map = {}
for name, group in ward_df.groupby('wd11cd'):
    ward_postcode_map[name] = list(group['pcds'])

/home/sagar/anaconda3/envs/env_py2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (5,8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [31]:
LSOA_savings = {}

for name, group in tqdm(non_generic_opioid.groupby('2')):
    total_saving = np.sum(group['23'])
    if name in LSOA_dist:        
        for k in LSOA_dist[name]:
            if k not in LSOA_savings:
                LSOA_savings[k] = 0.0
            LSOA_savings[k]+= float(total_saving)*float(LSOA_dist[name][k])

100%|██████████| 6683/6683 [00:04<00:00, 1420.44it/s]


In [32]:
%store -r Ward_prevalence
%store -r Ward_patients
%store -r DiseasePop_byWard

In [33]:
df = pd.read_csv('../BL_Work/Lower_Layer_Super_Output_Area_2011_to_Ward_2015_Lookup_in_England_and_Wales.csv')

In [34]:
wardMap = {}
for name , group in df.groupby('WD15CD'):
        wardMap[name] = list(group['LSOA11CD'])

In [35]:
boroughMap = {}
for name , group in df.groupby('LAD15NM'):
        boroughMap[name] = list(group['LSOA11CD'])

In [36]:
population_borough = {}
for name,group in ward_pop.groupby('Local Authority'):
    population_borough[name] = 0.0
    for index , row in group.iterrows():
        population_borough[name] += float(row['All Ages'].replace(',',''))

In [37]:
borough_savings_UK = {}

for k in boroughMap:
    try:
        borough_savings= np.sum([LSOA_savings[w] for w in boroughMap[k] if not np.isnan(LSOA_savings[w])])
        pop = population_borough[k]
        borough_savings_UK[k] = (float(borough_savings)/float(pop))*1000.0
        
    except:
        print "Some ward had wrong data"

Some ward had wrong data
Some ward had wrong data
Some ward had wrong data
Some ward had wrong data
Some ward had wrong data
Some ward had wrong data
Some ward had wrong data
Some ward had wrong data
Some ward had wrong data
Some ward had wrong data
Some ward had wrong data
Some ward had wrong data
Some ward had wrong data
Some ward had wrong data
Some ward had wrong data
Some ward had wrong data
Some ward had wrong data
Some ward had wrong data
Some ward had wrong data
Some ward had wrong data
Some ward had wrong data
Some ward had wrong data


In [38]:
borough_savings_UK

{'Adur': 62.382589254076784,
 'Allerdale': 52.999101666727086,
 'Amber Valley': 31.826500681836762,
 'Arun': 56.775716733646696,
 'Ashfield': 26.15528655965845,
 'Ashford': 97.50052533207403,
 'Aylesbury Vale': 42.59424558225823,
 'Babergh': 101.01539683949332,
 'Barking and Dagenham': 24.791210558184464,
 'Barnet': 22.043417799024187,
 'Barnsley': 40.46808962976673,
 'Barrow-in-Furness': 90.96238808002161,
 'Basildon': 33.70749875446775,
 'Basingstoke and Deane': 56.99035536979711,
 'Bassetlaw': 43.027135090066906,
 'Bath and North East Somerset': 24.595401506295904,
 'Bedford': 51.118521722809376,
 'Bexley': 49.06857874102955,
 'Birmingham': 36.32051148580374,
 'Blaby': 51.062330469577056,
 'Blackburn with Darwen': 43.37247602954727,
 'Blackpool': 66.77493816667376,
 'Bolsover': 54.829024054267244,
 'Bolton': 70.15331556445814,
 'Boston': 96.52303561990868,
 'Bournemouth': 62.592885178114706,
 'Bracknell Forest': 38.504885027410594,
 'Bradford': 34.81222366739729,
 'Braintree': 62.31

In [39]:
files[:3]

['../BL_Work/openPrescribe/savings/201008.h5',
 '../BL_Work/openPrescribe/savings/201009.h5',
 '../BL_Work/openPrescribe/savings/201010.h5']

In [40]:
monthly_borough_savings = {}
monthly_borough_dosage = {}
monthly_borough_costs = {}
monthly_borough_items = {}

fail = 0
#For temporal secection, the pdp dataframe is useless. Change the file slicing here to allow different time frames.
# for f in tqdm(files[-3:]):
for f in tqdm(files):

    month = f.split('/')[-1].split('.')[0]
    monthly_borough_savings[month] = {}
    monthly_borough_items[month] = {}
    
    pdp = pd.read_hdf(f)
#     opioids = pdp.loc[pdp['16'].isin(opiod_BNF['BNF_code'])]
    #User prescriptions ome
    opioids = pdp[pdp['3'].isin(ome_map.keys())]
    opioids = calculateOME(opioids , ome_map) 
    non_generic_opioid = opioids.loc[opioids['20']!='AA']
    LSOA_savings = {}
    LSOA_dosage = {}
    LSOA_costs = {}
    LSOA_items = {}
    for name, group in opioids.groupby('2'):
#         total_dosage = np.sum(group['19'])
#         total_dosage = np.sum(group['24'])
        #Use prescription ome
        total_dosage = np.sum(group['presc_ome'])
        total_cost = np.sum(group['7'])
        total_items = np.sum(group['5'])
        if name in LSOA_dist:        
            for k in LSOA_dist[name]:
                if k not in LSOA_dosage:
                    LSOA_dosage[k] = 0.0
                    LSOA_costs[k] = 0.0
                    LSOA_items[k] = 0.0
                    
                LSOA_dosage[k]+= float(total_dosage)*float(LSOA_dist[name][k])
                LSOA_costs[k]+= float(total_cost)*float(LSOA_dist[name][k])
                LSOA_items[k]+= float(total_items)*float(LSOA_dist[name][k])
    
    for name, group in non_generic_opioid.groupby('2'):
        total_saving = np.sum(group['23'])
        if name in LSOA_dist:        
            for k in LSOA_dist[name]:
                if k not in LSOA_savings:
                    LSOA_savings[k] = 0.0
                LSOA_savings[k]+= float(total_saving)*float(LSOA_dist[name][k])
    
    borough_savings_UK = {}
    borough_dosage_UK = {}
    borough_cost_UK = {}
    borough_items_UK = {}
    for k in boroughMap:
        try:
            borough_savings= np.sum([LSOA_savings[w] for w in boroughMap[k] if not np.isnan(LSOA_savings[w])])
            borough_dosages= np.sum([LSOA_dosage[w] for w in boroughMap[k] if not np.isnan(LSOA_dosage[w])])
            borough_costs= np.sum([LSOA_costs[w] for w in boroughMap[k] if not np.isnan(LSOA_costs[w])])
            borough_items = np.sum([LSOA_items[w] for w in boroughMap[k] if not np.isnan(LSOA_items[w])])
            
            pop = population_borough[k]
            borough_savings_UK[k] = float(borough_savings)
            borough_dosage_UK[k] = (float(borough_dosages)/float(pop))*1000.0
            borough_cost_UK[k] = (float(borough_costs)/float(pop))*1000.0
            borough_items_UK[k] = (float(borough_items)/float(pop))*1000.0

        except:
            fail+=1
    
    monthly_borough_savings[month]  = borough_savings_UK
    monthly_borough_dosage[month]  = borough_dosage_UK
    monthly_borough_costs[month] = borough_cost_UK
    monthly_borough_items[month] = borough_items_UK

  0%|          | 0/107 [00:00<?, ?it/s]/10TBdrive/sagarj/anaconda3/envs/env_py2/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
100%|██████████| 107/107 [1:25:27<00:00, 49.00s/it]


In [41]:
UKwide_opioid = {}
UKUKwide_opioid_costs = {}
for k in monthly_borough_savings:
    UKwide_opioid[k] = sum(monthly_borough_savings[k].values())
for k in monthly_borough_costs:
    total = []
    for b in monthly_borough_costs[k]:
        scale = float( population_borough[b])/1000.0
        total.append(monthly_borough_costs[k][b]*scale)
    UKUKwide_opioid_costs[k] = sum(total)

In [42]:
monthly_borough_savings_opioid = monthly_borough_savings
monthly_borough_dosage_opioid = monthly_borough_dosage
monthly_borough_costs_opioid = monthly_borough_costs
monthly_borough_items_opioid = monthly_borough_items

In [43]:
# overspend_opioid = {}
# overspend_opioid['savings'] = UKwide_opioid
# overspend_opioid['costs'] = UKUKwide_opioid_costs

overspend_opioid = {}
overspend_opioid['items'] = monthly_borough_items_opioid
overspend_opioid['dosage'] = monthly_borough_dosage_opioid

In [44]:
# %store -r overspend_opioid

In [45]:
# monthly_borough_dosage_opioid

In [46]:
# %store monthly_borough_savings_opioid
# %store monthly_borough_dosage_opioid
# %store monthly_borough_costs_opioid


# %store -r monthly_borough_savings_opioid
# %store -r monthly_borough_dosage_opioid
# %store -r monthly_borough_costs_opioid

In [47]:
# json.dump(overspend_opioid , open("2019_monthly_borough_dosage_items_opioid.json","w"))

In [48]:
dosage_2019 = json.load(open("2019_monthly_borough_dosage_items_opioid.json","r"))
dosage_2010 = json.load(open("2010_monthly_borough_dosage_items_opioid.json","r"))

In [49]:
# dosage_2019['items']

In [50]:
# dosage_2010['items']

In [51]:
# Between 2010 and 2019, there was a X% increase in opioid items prescribed (from X per 1000 to Y per 1000 population). 
# However, after correcting for OME, the total volume prescribed increased by YY% (from X mg to Y mg OME per 1000 population).”
dosage = []
for month in dosage_2019['items']:
    yearDose = []
    for area in dosage_2019['items'][month]:
        yearDose.append(dosage_2019['items'][month][area])
    dosage.append(np.mean(yearDose))
print(4*np.sum(dosage))



dosage = []
for month in dosage_2010['items']:
    yearDose = []
    for area in dosage_2010['items'][month]:
        yearDose.append(dosage_2010['items'][month][area])
    dosage.append(np.mean(yearDose))
print(4*np.sum(dosage))

423.24870129335994
292.1337831981477


In [52]:
dosage_2010['dosage'].keys()

[u'201010', u'201009', u'201008']

In [53]:
# Between 2010 and 2019, there was a X% increase in opioid items prescribed (from X per 1000 to Y per 1000 population). 
# However, after correcting for OME, the total volume prescribed increased by YY% (from X mg to Y mg OME per 1000 population).”
dosage = []
for month in dosage_2019['dosage']:
    yearDose = []
    for area in dosage_2019['dosage'][month]:
        yearDose.append(dosage_2019['dosage'][month][area])
    dosage.append(np.mean(yearDose))
print(4*np.sum(dosage))



dosage = []
for month in dosage_2010['dosage']:
    yearDose = []
    for area in dosage_2010['dosage'][month]:
        yearDose.append(dosage_2010['dosage'][month][area])
    dosage.append(np.mean(yearDose))
print(4*np.sum(dosage))

359027.91357510514
236510.07033414702


In [54]:
# %store -r monthly_borough_savings

In [55]:
IMD_District = json.load(open('scratchpad/District_IMD_scores_life_values.json','rb'))

In [56]:
IMD_District

{u'Adur': {u'District Code': [u'E07000223'],
  u'IMD': {u'mean_values': {u'Barriers to Housing and Services Score': 17.706571428571433,
    u'Crime Score': -0.1049047619047619,
    u'Education, Skills and Training Score': 28.01047619047619,
    u'Employment Score (rate)': 0.10511904761904763,
    u'Health Deprivation and Disability Score': -0.005071428571428567,
    u'Income Score (rate)': 0.12354761904761905,
    u'Living Environment Score': 18.868476190476194},
   u'median_values': {u'Barriers to Housing and Services Score': 17.7195,
    u'Crime Score': 0.030500000000000003,
    u'Education, Skills and Training Score': 26.0565,
    u'Employment Score (rate)': 0.086,
    u'Health Deprivation and Disability Score': -0.0365,
    u'Income Score (rate)': 0.10250000000000001,
    u'Living Environment Score': 17.901}},
  u'Life Values': {u'Anxiety': -1,
   u'Happiness': -1,
   u'Life Satisfaction': 56.42,
   u'Worthwhile': 45.7},
  u'aggregate_IMD': 18.275,
  u'pop_density': 1466.0302032741

In [57]:
savings = {}
for month in monthly_borough_savings:
    for dist in monthly_borough_savings[month]:
        if dist not in savings:
            savings[dist] = 0.0
        else:
            savings[dist]+=monthly_borough_savings[month][dist]
dose = {}
for month in monthly_borough_dosage:
    for dist in monthly_borough_dosage[month]:
        if dist not in dose:
            dose[dist] = 0.0
        else:
            dose[dist]+=monthly_borough_dosage[month][dist]

cost = {}
for month in monthly_borough_costs:
    for dist in monthly_borough_costs[month]:
        if dist not in cost:
            cost[dist] = 0.0
        else:
            cost[dist]+=monthly_borough_costs[month][dist]

            
dict_saving = {'Name':[],'Savings':[] , 'Population' :[] ,'Dosage':[] , 'Density' : [] , 'Seniors' : [] , 'IMD' : [] , 'TotalCost':[]}
for s in savings:
    dict_saving['Name'].append(s)
    dict_saving['Savings'].append(savings[s])
    dict_saving['TotalCost'].append(cost[s])
    dict_saving['Dosage'].append(dose[s])
    dict_saving['Density'].append(IMD_District[s]['pop_density'])
    dict_saving['Population'].append(IMD_District[s]['population'])
    dict_saving['Seniors'].append(IMD_District[s]['senior_pop'])
    dict_saving['IMD'].append(IMD_District[s]['aggregate_IMD'])



savings_opioid_df = pd.DataFrame(dict_saving)

In [ ]:
# % store -r savings_opioid_df

In [ ]:
savings_opioid_df['Savings_norm'] = (savings_opioid_df['Savings']/savings_opioid_df['Population'])*1000

In [ ]:
england = gp.read_file('../BL_Work/Local_Authority_Districts_December_2017_Ultra_Generalised_Clipped_Boundaries_in_United_Kingdom_WGS84.shp')

In [ ]:
england.head()

In [ ]:
merged = england.set_index('lad17nm').join(savings_opioid_df.set_index('Name'))
merged = merged[np.isfinite(merged['Savings'])]

In [ ]:
merged

In [ ]:
9 * np.sum(merged['Savings'])

In [ ]:
# merged.to_csv('Opioid_borough_All_data_ome_dosage.csv')

In [ ]:
# savings_opioid_df.head()

In [ ]:
savings_opioid_df['Dosage_total'] = (savings_opioid_df['Dosage']/1000)*savings_opioid_df['Population']

In [ ]:
from scipy.stats import pearsonr

pearsonr(savings_opioid_df['Dosage_total'] , savings_opioid_df['Density'])

In [ ]:
sorted_savings = merged.sort_values('Savings')

top = sorted_savings[-5:]
bottom = sorted_savings[:5]

In [ ]:
top

In [ ]:
bottom

In [ ]:
london_boroughs = pd.read_csv('scratchpad/london-borough-profiles.csv')
london_boroughs.head()

In [ ]:
london_boroughson_merged = pd.merge(merged , london_boroughs , left_on='lad17cd' , right_on='Code')

In [ ]:
london_boroughson_merged.head()

In [ ]:
print(list(london_boroughson_merged.columns))

In [ ]:
selected_dims = ['Dosage', 'IMD', 'Density','Average_Age,_2017', '%_of_resident_population_born_abroad_(2015)', '%_of_population_from_BAME_groups_(2016)',
                 'Unemployment_rate_(2015)', 'Youth_Unemployment_(claimant)_rate_18-24_(Dec-15)', 
                'Crime_rates_per_thousand_population_2014/15','Median_House_Price,_2015', '%_of_area_that_is_Greenspace,_2005',  
                'Life_satisfaction_score_2011-14_(out_of_10)', 'Worthwhileness_score_2011-14_(out_of_10)', 'Happiness_score_2011-14_(out_of_10)', 'Anxiety_score_2011-14_(out_of_10)', 'Childhood_Obesity_Prevalance_(%)_2015/16','Political_control_in_council', 'Proportion_of_seats_won_by_Conservatives_in_2014_election', 'Proportion_of_seats_won_by_Labour_in_2014_election', 'Proportion_of_seats_won_by_Lib_Dems_in_2014_election', 'Turnout_at_2014_local_elections']

In [ ]:
selected_london = london_boroughson_merged[selected_dims]
import matplotlib.pyplot as plt
corr = selected_london.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)
# plt.matshow()
# plt.show()

In [ ]:
sorted_dosage = london_boroughson_merged.sort_values('Dosage')

top_dosage = sorted_dosage[-4:]
bottom_dosage = sorted_dosage[:4]

In [ ]:
# set a variable that will call whatever column we want to visualise on the map
variable = 'Dosage'
vmin, vmax = min(london_boroughson_merged['Dosage']) , max(london_boroughson_merged['Dosage'])

fig, ax = plt.subplots(1, figsize=(18, 12))
london_boroughson_merged.plot(column=variable, cmap='Reds', linewidth=0.8, ax=ax, edgecolor='1')
ax.axis('off')

# top_dosage.plot(column=variable, cmap='Purples', linewidth=0,alpha=1 , ax=ax, edgecolor='White')

# bottom_dosage.plot(column=variable, cmap='Greens', linewidth=0,alpha=1 , ax=ax, edgecolor='Green')

    
# ax.set_title('Overspending for opioids', fontdict={'fontsize': 25, 'fontweight' : 5})
sm = plt.cm.ScalarMappable(cmap='Reds', norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []

cbar = fig.colorbar(sm,shrink=0.5)
cbar.ax.tick_params(labelsize=20)
cbar.set_label('$\widetilde u(a)$', fontsize = 20)

In [ ]:
top_dosage['Area_name']

In [ ]:
bottom_dosage['Area_name']

In [ ]:
# set a variable that will call whatever column we want to visualise on the map
variable = 'Savings_norm'
vmin, vmax = min(savings_opioid_df['Savings_norm']) , max(savings_opioid_df['Savings_norm'])

fig, ax = plt.subplots(1, figsize=(18, 12))
merged.plot(column=variable, cmap='Greys', linewidth=0.8, ax=ax, edgecolor='1')
ax.axis('off')

top.plot(column=variable, cmap='Reds', linewidth=0,alpha=1 , ax=ax, edgecolor='Red')

bottom.plot(column=variable, cmap='Greens', linewidth=0,alpha=1 , ax=ax, edgecolor='Green')

    
# ax.set_title('Overspending for opioids', fontdict={'fontsize': 25, 'fontweight' : 5})
sm = plt.cm.ScalarMappable(cmap='Greys', norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []

cbar = fig.colorbar(sm,shrink=0.5)
cbar.ax.tick_params(labelsize=20)
cbar.set_label('$\widetilde s_{ub}(a)$', fontsize = 20)

In [ ]:
sorted_dosage = merged.sort_values('Dosage')

top_dosage = sorted_dosage[-5:]
bottom_dosage = sorted_dosage[:5]

In [ ]:
top_dosage

In [ ]:
bottom_dosage

In [ ]:
# set a variable that will call whatever column we want to visualise on the map
variable = 'Dosage'
vmin, vmax = min(savings_opioid_df['Dosage']) , max(savings_opioid_df['Dosage'])

fig, ax = plt.subplots(1, figsize=(18, 12))
merged.plot(column=variable, cmap='Greys', linewidth=0.8, ax=ax, edgecolor='1')
ax.axis('off')

top_dosage.plot(column=variable, cmap='Reds', linewidth=0,alpha=1 , ax=ax, edgecolor='Red')

bottom_dosage.plot(column=variable, cmap='Greens', linewidth=0,alpha=1 , ax=ax, edgecolor='Green')

# ax.set_title('Normalized dosages per 1000', fontdict={'fontsize': 25, 'fontweight' : 5})
sm = plt.cm.ScalarMappable(cmap='Greys', norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []

cbar = fig.colorbar(sm,shrink=0.5)
cbar.ax.tick_params(labelsize=20)
cbar.set_label('$\widetilde u(a)$', fontsize = 20)

In [ ]:
savings_opioid_df[savings_opioid_df['Dosage'] == max(savings_opioid_df['Dosage'])]

In [ ]:
savings_opioid_df[savings_opioid_df['Savings'] == max(savings_opioid_df['Savings'])]

In [ ]:
standardized_merged = merged.copy()

In [ ]:
def normalize(df,feature_names):
    result = df.copy()
    for feature_name in feature_names:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [ ]:
standardized_merged = normalize(merged , ['Savings','Density','Dosage','IMD','Seniors'])

In [ ]:
standardized_merged.head()

In [ ]:
corr = pearsonr(standardized_merged['Dosage'], standardized_merged['IMD'])

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(12, 8)
sns.regplot(x='Dosage',y='IMD',data=standardized_merged)
ax.set_title("IMD Vs Impact dosage" , fontsize=20)
ax.set_xlabel("Impact Dosage",fontsize=20)
ax.set_ylabel("Normalized IMD",fontsize=20)
ax.annotate("Corr: " +  str(corr[0])[:4],  xy=(0.8, 0.2),fontsize = 18)


In [ ]:
corr = pearsonr(standardized_merged['Savings'], standardized_merged['IMD'])

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(12, 8)
sns.regplot(x='Savings',y='IMD',data=standardized_merged)
ax.set_title("IMD Vs Overspending: Anti Depressants" , fontsize=20)
ax.set_xlabel("Normalized Overspending",fontsize=20)
ax.set_ylabel("Normalized IMD",fontsize=20)
ax.annotate("Corr: " +  str(corr[0])[:4],  xy=(0.8, 0.2),fontsize = 18)


In [ ]:
corr = pearsonr(standardized_merged['Savings'], standardized_merged['Seniors'])

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(12, 8)
sns.regplot(x='Savings',y='Seniors',data=standardized_merged)
ax.set_title("Senior population Vs Savings from Generics: Opioids" , fontsize=20)
ax.set_xlabel("Normalized Savings",fontsize=20)
ax.set_ylabel("Normalized Senior population",fontsize=20)
ax.annotate("Corr: " +  str(corr[0])[:4],  xy=(0.8, 0.2),fontsize = 18)

In [ ]:
corr=pearsonr(standardized_merged['Savings'], standardized_merged['Density'])

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(12, 8)
sns.regplot(x='Savings',y='Density',data=standardized_merged)
ax.set_title("Population density Vs Savings from generics: Opioids" ,fontsize=20)
ax.set_xlabel("Normalized Savings from generics",fontsize=20)
ax.set_ylabel("Normalized population density",fontsize=20)
ax.annotate("Corr: " +  str(corr[0])[:4],  xy=(0.8, 0.8),fontsize = 18)

In [ ]:
corr=pearsonr(standardized_merged['Dosage'], standardized_merged['Density'])

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(12, 8)
sns.regplot(x='Dosage',y='Density',data=standardized_merged)
ax.set_title("Population density Vs Relative Normalized dosage for opioids" ,fontsize=20)
ax.set_xlabel("Relative normalized  dosage",fontsize=20)
ax.set_ylabel("Normalized population density",fontsize=20)
ax.annotate("Corr: " +  str(corr[0])[:4],  xy=(0.8, 0.8),fontsize = 18)


In [ ]:
pearsonr(standardized_merged['Dosage'], standardized_merged['Seniors'])

In [ ]:
pearsonr(standardized_merged['Dosage'], standardized_merged['Density'])

In [ ]:
sns.regplot(x='Savings',y='Seniors',data=standardized_merged)

In [ ]:
monthlySavings = {}
for k in monthly_borough_savings:
    monthlySavings[k] = np.median(monthly_borough_savings[k].values())
    
    
monthlyCosts = {}
for k in monthly_borough_costs:
    monthlyCosts[k] = np.median(monthly_borough_costs[k].values())